# Plot Concentration Bound
---

## Geometry of $\mathcal{X}\times \mathcal{Y}$: 
### Set the Rate at Which Embedding Distorts Geometry in As a Function of Euclidean Dimension

We consider four possible cases (``distortion_type``):
1. Worst-case (expander graph-type geometries)
2. Polynomial embedding (trees).
3. Subsets of Euclidean space.
4. Subspaces (metric) of the real line.

In [10]:
distortion_type = 3

### Decide on N vs k

### N
Once the number of training samples, $N$, is set then, we consider two scenarios:
- if ``N_is_Small==True``: $N$ ranges from $\lfloor N/10 \rfloor$ to $N$
- if ``N_is_Small==True``: $N$ ranges from $N^3$ to $N^4$

In [11]:
N = 1*(10**4)
N_is_Small = False

### k

#### Either we set k manually or we consider the no. Points Needed to Pack a Cube:

From [George G. Lorentz, Manfred v. Golitschek, and Yuly Makovoz. Constructive approximation - Proposition 1.3](https://link.springer.com/book/9783642646102) the packing number of $[0,\operatorname{diam}]^d$ in $\ell_{\infty}^d$ by $\ell_{\infty}^d$-balls of radius $10^{-p}$ is no more than
$$
k \ge 
\Big\lceil
2^{-d}
        \,
        \Big(10^{p}*\operatorname{diam}\Big)^d
\Big\rceil
$$
we use this lower-bound to compute $k$.

In [12]:
# Cube or no Cube?
cube_k = True
# If Manual 
k_manual = 1*(10**8)

# If Cube
diam = 1*(10**0)
d = 1*(10**1); print('Prototypical Space: [0,'+str(diam)+']^'+str(d))
p = 1*(10**1); print('Distance Between Points on the Grid: '+str(10**-p))
# Use less points (w.o. 2^-d factor)
use_no_2d_factor = True

%run ./SupportFiles/Compute_N_points.ipynb

Prototypical Space: [0,1]^10
Distance Between Points on the Grid: 1e-10
Number of points:9.77e+96


## Import Packages and Set Global Hyperparameters

In [13]:
%run ./SupportFiles/Dependancies.ipynb
print('---')

Dependancies and Packages Loaded
---


# Set Global Parameters

## Are we Computing Risk Bounds or Concentration Bounds (in $\mathcal{W}_1$)?
- For Risk Bounds set: True
- For Concentration Bounds set: False

In [5]:
Risk_or_Concentration = True
Isometric_Embedding = True

#### Decide on How Many Points to Use?

Load Global hyperparameters, now that internal parameters have been defined.

In [ ]:
%run ./SupportFiles/Global_Hyperparameters.ipynb
print('---')

---

Load helper functions.

In [ ]:
print('---')
%run ./SupportFiles/Helper_Functions.ipynb
print('---')

# The Risk Bound Computers

We compare our risk bound against the following binary classification benchmarks.

## Benchmark: Trivial VC/Occam Bound (Binary Classification)
From the The (Quantitative) Fundamental Theorem of Statistical Learning ([Theorem 6.8 in "Understanding Machine Learning" (with extra details in Section 28.1)](https://scholar.google.com/citations?view_op=view_citation&hl=en&user=kezPqwoAAAAJ&citation_for_view=kezPqwoAAAAJ:XiVPGOgt02cC)
$$
\begin{aligned}
    \mathcal{R}(f) - \hat{\mathcal{R}}(f)
\le &
    n^{-1/2}
    (
        128 \operatorname{VC-dim}\log(n+1) + \log(8/\delta)
    )^{1/2}
\\ 
\le & 
    n^{-1/2}
    (
        128 \#\mathcal{X}\log(n+1) + \log(8/\delta)
    )^{1/2}
\end{aligned}
$$
where we use the fact that $\operatorname{VC-dim}(2^{\mathcal{X}})\le \log_2(\#2^{\mathcal{X}})=\#\mathcal{X}$.

## Hoeffding Bound


From [Corollary 4.6 of "Understanding Machine Learning"](https://scholar.google.com/citations?view_op=view_citation&hl=en&user=kezPqwoAAAAJ&citation_for_view=kezPqwoAAAAJ:XiVPGOgt02cC) we have the following Heofding-type bound
$$
\begin{aligned}
    (2n)^{-1/2}
    \log\big(
    2 \times 2^k/\delta
    \big)^{1/2}
    = 
    (2n)^{-1/2}
     \big(
     \log(2/\delta) + k\log(2)
     \big)^{1/2}
\end{aligned}
$$

Load risk bound functions.

In [ ]:
%run ./SupportFiles/Risk_Bound_Engine.ipynb

## Generate Plot

#### Decide on Scale of Sample Size (Range: Small N or Large N)

In [ ]:
# Indices
Freq_plot = 100
indexing_set = range(1,10000,Freq_plot)

In [ ]:
Print_Occam = False

---

In [ ]:
ax = plt.figure(figsize=(10,10))
sns.set_style("darkgrid")

plt.title(''+dist_name+', k='+str('{:.2e}'.format(k))+', Varying F and N')
    

plt.xlabel('No. Samples (N)')
plt.ylabel('Risk Bound')# on $\sup_{f\in \mathcal{F}}\,|\mathcal{R}(\hat{f})-\hat{\mathcal{R}}(\hat{f})|$')
# plt.ylabel('Bound on $\mathbb{E}[\mathcal{W}_1(\mathbb{P},\mathbb{P}^N)]$')



# Set Max No Colours
n_col=max(indexing_set)+2


#### Plot Generation
for i in indexing_set:
    # Set Value of F
    F_global = int(np.round(i,0))
    
    # Write Function of N only
    def get_risk_bound__NVar_scalar(N):
        return get_risk_bound__scalar(k=k_global,F=F_global,N=N)
    
    get_risk_bound__NVar = np.vectorize(get_risk_bound__NVar_scalar)
    
    # Compute Risk Bound
    Risk_Bounds = get_risk_bound__NVar_scalar(N_sequence)

    
    # Plot
    col_index = (i-min(indexing_set))/(max(indexing_set)+1)
    # Plot Risk-Bound
    if i % (Freq_plot*10) == 1: # Ony write every other legend
        plt.plot(N_sequence, 
             Risk_Bounds,
             color=colorFader(c1,c2,col_index),
             label = 'F='+str(F_global).format(i=i))
    else: # Remove legend on most plots
        plt.plot(N_sequence, 
             Risk_Bounds,
             color=colorFader(c1,c2,col_index),label='_nolegend_')


### Classical Risk-Bounds for Binary Classification
if Y_is_bin_class == True:
    if Print_Occam:
        ###----------------###
        ### VC/Occam Bound ###
        ###----------------###
        # Write Function of N only
        def get_risk_bound__VC_scalar(N):
            return trivial_VCBound(N_in=N,k_in=k,delta = delta)
        get_risk_bound__VC = np.vectorize(get_risk_bound__VC_scalar)

        # Compute Risk Bound
        Risk_Bounds__VC = get_risk_bound__VC(N_sequence)

        # Plot Risk-Bound
        plt.plot(N_sequence,Risk_Bounds__VC,label = 'Occam',c='orange',linewidth=3)
    ###-----------------###
    ### Hoeffding Bound ###
    ###-----------------###
    # Write Function of N only
    def get_risk_bound__Hoeffding_scalar(N):
        return Hoeffding_Bound(N_in=N,k_in=k,delta = delta)
    get_risk_bound__Hoeffding = np.vectorize(get_risk_bound__Hoeffding_scalar)
    
    # Compute Risk Bound
    Risk_Bounds__Hoeffding = get_risk_bound__Hoeffding(N_sequence)
    
    # Plot Risk-Bound
    plt.plot(N_sequence,Risk_Bounds__Hoeffding,label = 'Hoeffding',c='red',linewidth=3)

# plt.legend(loc=0) #'best'
plt.legend(loc=1) #'Upper right'
# plt.legend(loc=2) # 'Upper left'


# ------------------- #
# Our Risk-Bound Only #
# ------------------- #
sns.set_style("whitegrid")
axes1 = ax.add_axes([0.2, 0.2, 0.3, 0.3]) # inset axes (location first, size second)
# axes1.set_title('Bound',fontsize=15)

#### Plot Generation
for i in indexing_set:
    # Set Value of F
    F_global = int(np.round(i,0))
    
    # Write Function of N only
    def get_risk_bound__NVar_scalar(N):
        return get_risk_bound__scalar(k=k_global,F=F_global,N=N)
    
    get_risk_bound__NVar = np.vectorize(get_risk_bound__NVar_scalar)
    
    # Compute Risk Bound
    Risk_Bounds = get_risk_bound__NVar_scalar(N_sequence)

    
    # Plot
    col_index = (i-min(indexing_set))/(max(indexing_set)+1)
    plt.plot(N_sequence, Risk_Bounds,label = 'F='+str(F_global).format(i=i),color=colorFader(c1,c2,col_index))

if Y_is_bin_class == True:
    ###-----------------###
    ### Hoeffding Bound ###
    ###-----------------###
    # Write Function of N only
    def get_risk_bound__Hoeffding_scalar(N):
        return Hoeffding_Bound(N_in=N,k_in=k,delta = delta)
    get_risk_bound__Hoeffding = np.vectorize(get_risk_bound__Hoeffding_scalar)
    
    # Compute Risk Bound
    Risk_Bounds__Hoeffding = get_risk_bound__Hoeffding(N_sequence)
    
    # Plot Risk-Bound
    plt.plot(N_sequence,Risk_Bounds__Hoeffding,label = 'Hoeffding',c='red',linewidth=3)

# ---------------------------------------- #
# High Representation Space Dimension Only #
# ---------------------------------------- #
sns.set_style("ticks")
axes2 = ax.add_axes([0.44, 0.55, 0.3, 0.3]) # inset axes (location first, size second)
axes2.set_title('Large Rep. Dim.',fontsize=15)

#### Plot Generation
for i in indexing_set[-int(len(indexing_set)/10):]:
    # Set Value of F
    F_global = int(np.round(i,0))
    
    # Write Function of N only
    def get_risk_bound__NVar_scalar(N):
        return get_risk_bound__scalar(k=k_global,F=F_global,N=N)
    
    get_risk_bound__NVar = np.vectorize(get_risk_bound__NVar_scalar)
    
    # Compute Risk Bound
    Risk_Bounds = get_risk_bound__NVar_scalar(N_sequence)

    
    # Plot
    col_index = (i-min(indexing_set))/(max(indexing_set)+1)
    plt.plot(N_sequence, Risk_Bounds,label = 'F='+str(F_global).format(i=i),color=colorFader(c1,c2,col_index))

    
    
plt.xlabel('No. Samples (N)',fontsize=10)
plt.ylabel('Risk Bound',fontsize=10)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# ----------------------------------------------------- #

plt.savefig('Experiments/Risk_Bound_F/WBenchmarks__NF_kis'+str(k_global)+'____'+str(dist_name)+'.pdf')
plt.show()

---
# Fin #
---

---

---